<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2024/aoc2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Global generic utils

In [111]:
from collections import Counter
from functools import reduce
from itertools import groupby, islice, takewhile

def vec_add(v1, v2):
  return tuple(sum(i) for i in zip(v1, v2))

def vec_sub(v1, v2):
  return tuple(i[0] - i[1] for i in zip(v1, v2))

dirs45 = [(dy, dx) for dy in [-1, 0, 1] for dx in [-1, 0, 1] if dy != 0 or dx != 0]

dir_up = (-1, 0)
dir_right = (0, 1)
dir_down = (1, 0)
dir_left = (0, -1)

dirs90 = [dir_up, dir_right, dir_down, dir_left]
turn_right = {dir_up: dir_right, dir_right: dir_down, dir_down: dir_left, dir_left: dir_up}
turn_left = {dir_up: dir_left, dir_left: dir_down, dir_down: dir_right, dir_right: dir_up}

def in_range(p, dimensions):
  y, x = p
  height, width = dimensions
  return y >= 0 and y < height and x >= 0 and x < width

def get_cell(p, grid):
  y, x = p
  return grid[y][x]

# use with itertools islice(limit), islice(start, limit, step), takewhile(predicate, iterate(...))
def iterate(start, func):
  current = start
  while True:
    yield current
    current = func(current)

def read_grid(filename):
  input = [line.strip() for line in open(f'drive/MyDrive/AoC/2024/{filename}.txt')]
  height = len(input)
  width = len(input[0])
  return input, height, width

Day 01, input

In [ ]:
input01 = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input01.txt')]

Day 01, part 1, sum of absolute diff of sorted list elements

In [ ]:
sum(abs(l - r) for (l, r) in zip(sorted(l for (l, _) in input01), sorted(r for (_, r) in input01)))

1530215

Day 01, part 2, sum lhs elements multiplied by frequency in rhs list

In [ ]:
from collections import Counter
r_counts = Counter(r for (_, r) in input01)
sum(l * r_counts[l] for (l, _) in input01)

26800609

Day 02, input

In [ ]:
input02 = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input02.txt')]

Day 02, utils

In [ ]:
def same_sgn(deltas_line):
  return all(deltas_line[i] * deltas_line[i+1] > 0 for i in range(len(deltas_line)-1))

def delta_max(deltas_line, m):
  return all(abs(d) <= m for d in deltas_line)

Day 02, part 1, determine (all increasing or all decreasing) and deltas < 4

In [ ]:
deltas = [[line[n+1] - line[n] for n in range(len(line)-1)] for line in input02]

sum(1 for deltas_line in deltas if same_sgn(deltas_line) and delta_max(deltas_line, 3))

359

Day 02, part 2, allow removal of any single element

In [ ]:
def check_line(line):
  deltas = [line[i+1] - line[i] for i in range(len(line)-1)]
  return same_sgn(deltas) and delta_max(deltas, 3)

def remove1(line):
  return [line[:i] + line[i+1:] for i in range(len(line))]

# part 1 regression
# print(sum(1 for line in input02 if check_line(line)))

# part 2
sum(1 for line in input02 if check_line(line) or any(check_line(r) for r in remove1(line)))

418

Day 03, input

In [ ]:
input03 = "".join(line for line in open('drive/MyDrive/AoC/2024/input03.txt'))

Day 03, utils

In [ ]:
import re

def find_mul_and_eval(instructions):
  # regex mul\\((\\d+),(\\d+)\\)
  matches = re.findall("mul\((\d+),(\d+)\)", instructions)
  return sum(int(x) * int(y) for (x, y) in matches)

Day 03, part 1, find mul(x,y) sequences, multiply and add

In [ ]:
# input03 = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
find_mul_and_eval(input03)

162813399

Day 03, part 2, evaluate do() and don't() sequences

In [ ]:
import re
from functools import reduce

def next_state(state, split):
  s, is_active = state
  match split:
    case "do()":
      return (s, True)
    case "don't()":
      return (s, False)
    case _:
      return (s + (find_mul_and_eval(split) if is_active else 0), is_active)

# input03 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
# split on (do\\(\\)|don't\\(\\)) - capturing group leads to delimiters being included in result
s, is_active = reduce(next_state, re.split("(do\(\)|don't\(\))", input03), (0, True))
s

53783319

Day 04, input

In [113]:
input04, rows, cols = read_grid("input04")

Day 04, part 1, find XMAS

In [114]:
def check_char(p, char):
  return in_range(p, (rows, cols)) and get_cell(p, input04) == char

def check(s, p, d):
  return s == "" or check_char(p, s[0]) and check(s[1:], vec_add(p, d), d)

def count_xmas(p):
  return sum(1 for d in dirs45 if check("XMAS", p, d))

sum(count_xmas((y, x)) for y in range(rows) for x in range(cols))

2567

Day 04, part 2, find
```
M.S
.A.
M.S
```



In [116]:
# The pattern can only occur in 4 different variants, with 'A' at (0,0)
patterns = [
  (((-1, -1), 'M'), ((-1, 1), 'S'), ((1, -1), 'M'), ((1, 1), 'S')),
  (((-1, -1), 'M'), ((-1, 1), 'M'), ((1, -1), 'S'), ((1, 1), 'S')),
  (((-1, -1), 'S'), ((-1, 1), 'M'), ((1, -1), 'S'), ((1, 1), 'M')),
  (((-1, -1), 'S'), ((-1, 1), 'S'), ((1, -1), 'M'), ((1, 1), 'M'))
]

# variant without range check
def check_char_unsafe(p, char):
  return get_cell(p, input04) == char

def check_pattern(p, pattern):
  return all(check_char_unsafe(vec_add(p, d), char) for (d, char) in pattern)

# Search for A within (1, 1)...(rows-1, cols-1) and check match for all patterns
sum(1
    for p in [(y, x) for y in range(1, rows-1) for x in range(1, cols-1) if get_cell((y, x), input04) == 'A']
    for pattern in patterns if check_pattern(p, pattern))

2029

Day 05, input

In [118]:
input05, _, _ = read_grid("input05")
split_index = input05.index('')
rules = [tuple(int(n) for n in rule.split('|')) for rule in input05[:split_index]]
updates = [[int(n) for n in update.split(',')] for update in input05[split_index+1:]]

Day 05, utils


In [119]:
def verify_pair(left, right):
  """if a rule exists, check order against it, otherwise pass"""
  existing_rules = [rule for rule in rules if left in rule and right in rule]
  return not existing_rules or any(rl == left and rr == right for rl, rr in existing_rules)

def verify_update(update):
  return all(verify_pair(update[i], update[j]) for i in range(len(update)-1) for j in range(i+1, len(update)))

Day 05, part 1, find correct updates, sum up middle their elements

In [120]:
sum(update[int(len(update)/2)] for update in updates if verify_update(update))

7365

Day 05, part 2, fix updates which break the rules, sum up their middle elements

In [121]:
def fix_update(update_rest, fixed):
  if not update_rest:
    return fixed
  if not fixed:
    return fix_update(update_rest[1:], [update_rest[0]])
  # find correct place for update_rest[0] in fixed
  for i in range(len(fixed)):
    if verify_pair(update_rest[0], fixed[i]):
      return fix_update(update_rest[1:], fixed[:i] + [update_rest[0]] + fixed[i:])
  return fix_update(update_rest[1:], fixed + [update_rest[0]])

sum(fix_update(update, [])[int(len(update)/2)] for update in updates if not verify_update(update))

5770

Day 06, input

In [123]:
input06, height, width = read_grid("input06")
dimensions = (height, width)
start = [(y, x) for y in range(height) for x in range(width) if get_cell((y, x), input06) == '^'][0]

Day 06, part 1, visisted cells

In [124]:
def get_visited(grid) -> tuple[list[tuple[tuple[int, int], tuple[int, int]]], bool]:
  dir = dir_up
  p = start
  visited = {(p, dir)}
  while in_range(p, dimensions):
    next = vec_add(p, dir)
    if in_range(next, dimensions):
      if get_cell(next, input06) == '#':
        dir = turn_right[dir]
      elif (next, dir) in visited:
        return (visited, True)
      else:
        p = next
        visited.add((p, dir))
    else:
      return (visited, False)

visited, _ = get_visited(input06)
len({p for (p, dir) in visited})

4711

Day 06, part 2, find loops

In [ ]:
mutable = [[c for c in row] for row in input06]
loops = 0
for y in range(height):
  # print(y, loops)
  for x in range(width):
    if get_cell((y, x), input06) == '.':
      mutable[y][x] = '#'
      _, loop = get_visited(mutable)
      loops += loop
      mutable[y][x] = '.'
loops

Day 07, input

In [ ]:
# [(desired_result, [operand1, operand2, ...])]
input07 = [
    (int(result.strip()), [int(operand.strip()) for operand in operands.split()])
    for (result, operands) in [[s.strip() for s in line.split(':')] for line in open('drive/MyDrive/AoC/2024/input07.txt')]]

Day 07, part 1, find valid equations with '*' and '+'

In [ ]:
def check_equation(desired_result, current_result, remaining_operands):
  if not remaining_operands:
    return desired_result == current_result
  if current_result > desired_result:
    return False
  return check_equation(desired_result, current_result + remaining_operands[0], remaining_operands[1:]) or \
         check_equation(desired_result, current_result * remaining_operands[0], remaining_operands[1:])

sum(desired_result for (desired_result, operands) in input07 if check_equation(desired_result, operands[0], operands[1:]))

4555081946288

Day 07, part 2, find valid equations with '*', '+', '||' (concat)

In [ ]:
from math import log10
def check_equation2(desired_result, current_result, remaining_operands):
  if not remaining_operands:
    return desired_result == current_result
  if current_result > desired_result:
    return False
  return check_equation2(desired_result, current_result + remaining_operands[0], remaining_operands[1:]) or \
         check_equation2(desired_result, current_result * remaining_operands[0], remaining_operands[1:]) or \
         check_equation2(desired_result, current_result * 10 ** (int(log10(remaining_operands[0])) + 1) + remaining_operands[0], remaining_operands[1:])
         # or with strings: check_equation2(desired_result, int(f'{current_result}{remaining_operands[0]}'), remaining_operands[1:])

sum(desired_result for (desired_result, operands) in input07 if check_equation2(desired_result, operands[0], operands[1:]))

227921760109726

Day 08, input

In [96]:
input08, height, width = read_grid("input08")
antennas = {}
for (name, pos) in [(input08[y][x], (y, x)) for y in range(height) for x in range(width) if input08[y][x] != '.']:
  if name in antennas:
    antennas[name].append(pos)
  else:
    antennas[name] = [pos]


Day 08, part 1, find antinodes (unique positions)

In [97]:
def antinodes(positions):
  return [p for p in [vec_sub(p1, vec_sub(p2, p1)) for p1 in positions for p2 in positions if p1 != p2] if in_range(p, dimensions)]

len({p for a in antennas.values() for p in antinodes(a)})

354

Day 08, part 2, find more antinodes (unique positions)


In [98]:
def antinodes2(positions):
  def antinodes_for_pair(p1, p2):
    anodes = []
    dir = vec_sub(p2, p1)  # step
    p = p2  # p1, p2 are antinodes themselves, but no need to add p1 as we also call this for the swapped pair
    while in_range(p, dimensions):
      anodes.append(p)
      p = vec_add(p, dir)
    return anodes
  return {a for p1 in positions for p2 in positions if p1 != p2 for a in antinodes_for_pair(p1, p2)}

len({a for p in antennas.values() for a in antinodes2(p)})

1263

Day 09, input

In [ ]:
input09 = open('drive/MyDrive/AoC/2024/input09.txt').read().strip()
# input09 = '2333133121414131402'  # example
codes = [int(c) for c in input09 + '0']

Day 09, part 1, fill free space with single blocks

In [ ]:
drive = [block for i in range(0, len(codes)-1, 2) for block in [int(i/2)] * codes[i] + [-1] * codes[i+1]]
i = 0
j = len(drive) - 1
compacted = []
while j >= i:
  if drive[i] >= 0:
    compacted.append(drive[i])
  else:
    compacted.append(drive[j])
    j -= 1
    while drive[j] == -1:
      j -= 1
  i += 1

sum(b * i for (b, i) in zip(compacted, range(len(compacted))))

6384282079460

Day 09, part 2, fill free space with whole files

In [ ]:
### SLOW for full input!!! ###
drive = [block for i in range(0, len(codes)-1, 2) for block in [int(i/2)] * codes[i] + [-1] * codes[i+1]]
compacted = [b for b in drive]
j = len(compacted) - 1
# print(compacted)
while j >= 0:
  while compacted[j] == -1:
    j -= 1
  # found number, determine size
  n = j
  while compacted[n] == compacted[j]:
    n -= 1
  num_size = j - n
  # look for gaps
  i = 0
  while (i < j):
    while compacted[i] != -1 and i < j:
      i += 1
    # found gap, determine size
    g = i
    while compacted[g] == -1:
      g += 1
    gap_size = g - i
    if (gap_size >= num_size):
      # move
      for k in range(num_size):
        compacted[i + k] = compacted[j - k]
        compacted[j - k] = -1
        # print(compacted)
      break
    i = g

  j = n

sum((b if b >= 0 else 0) * i for (b, i) in zip(compacted, range(len(compacted))))

2858

Day 10, input


In [ ]:
input10 = [[int(c) if c.isdigit() else -1 for c in line.strip()] for line in open('drive/MyDrive/AoC/2024/input10.txt')]
height = len(input10)
width = len(input10[0])
dimensions = (height, width)
trailheads = [(y, x) for y in range(height) for x in range(width) if get_cell((y, x), input10) == 0]

Day 10, depth first search for peaks (9)

In [ ]:
from functools import reduce
def find_peaks(p, visited, peaks):
  if not in_range(p, dimensions) \
    or get_cell(p, input10) == -1 \
    or p in visited \
    or visited and get_cell(p, input10) != get_cell(visited[-1], input10) + 1:
    return peaks
  if get_cell(p, input10) == 9:
    # print(visited + [p])
    return peaks + [p]
  return reduce(lambda ps, d: find_peaks(vec_add(p, d), visited + [p], ps), dirs90, peaks)


Day 10, part 1, sum number of peaks reachable per trailhead

In [ ]:
sum(len(set(find_peaks(p, [], []))) for p in trailheads)

796

Day 10, part 2, sum number of distinct trails to peaks per trailhead

In [ ]:
sum(len(find_peaks(p, [], [])) for p in trailheads)

1942

Day 11, input

In [82]:
input11 = [s.strip() for s in "8435 234 928434 14 0 7 92446 8992692".split()]
# input11 = [s.strip() for s in "125 17".split()]

Day 11, funcs

In [81]:
def blink_single(s):
  # if s in cache:
    # return cache[s]
  if s == '0':
    res = ['1']
  elif len(s) % 2 == 0:
    h = len(s) // 2
    res = [s[:h], f'{int(s[h:])}']
  else:
    res = [f'{int(s) * 2024}']
  #cache[s] = res
  return res

cache = {}
cache_hits = []
iter = {}
def blink(line, iterations):
  iter[iterations] = iter[iterations] + 1 if iterations in iter else 1
  if iterations == 0:
    return line
  res = []
  for i in range(len(line)):
    key = (line[i], iterations)
    if key in cache:
      cache_hits.append(key)
      b = cache[key]
    else:
      b = blink(blink_single(line[i]), iterations - 1)
      cache[key] = b
    res += b
  return res

Day 11, part 1, 25 itertions / part 2, 75 iterations

In [83]:
iter = {}
cache = {}
cache_hits = []
r = blink(input11, 25)
# print(r)
print(f'stones: {len(r)}')
print(f'blink() calls: {sum(i for i in iter.values())}, by iterations to go: {iter}')
print(f'cache entries: {len(cache)}, cache size (number of strings, regardless of length): {sum(len(v) for v in cache.values())}')
print(f'cache hits: {len(cache_hits)}, distinct: {len(set(cache_hits))}')

stones: 182081
blink() calls: 3645, by iterations to go: {25: 1, 24: 8, 23: 10, 22: 13, 21: 19, 20: 27, 19: 32, 18: 43, 17: 55, 16: 68, 15: 85, 14: 92, 13: 111, 12: 114, 11: 123, 10: 151, 9: 156, 8: 183, 7: 200, 6: 213, 5: 238, 4: 268, 3: 319, 2: 326, 1: 389, 0: 401}
cache entries: 3644, cache size (number of strings, regardless of length): 1493775
cache hits: 1533, distinct: 476


Day 12, input

In [92]:
grid, dimensions = read_lines("input12")
# grid=['AAAA','BBCD','BBCC','EEEC']
# grid=['OOOOO','OXOXO','OOOOO','OXOXO','OOOOO']
# dimensions=(len(grid), len(grid[0]))
width, height = dimensions

Day 12, part 1, areas * perimeters

In [94]:
def perimeter(p, grid):
  c = get_cell(p, grid)
  return sum(0 if in_range(vec_add(p, d), dimensions) and get_cell(vec_add(p, d), grid) == c else 1 for d in dirs90)

global_visited = set()

def region(p, grid):
  c = get_cell(p, grid)
  visited = set()

  def collect(p):
    if p in global_visited or p in visited or not in_range(p, dimensions) or get_cell(p, grid) != c:
      return
    visited.add(p)
    global_visited.add(p)
    for d in dirs90:
       collect(vec_add(p, d))

  collect(p)
  return c, visited

regions = [region(p, grid) for p in [(y, x) for y in range(height) for x in range(width)] if not p in global_visited]
# print(regions)

# len(r) = area of region r (consisting of all points for the region) with char c
sum(sum(perimeter(p, grid) for p in r) * len(r) for (c, r) in regions)


1494342

Day 12, part 2, areas * side count